# Load Libraries

In [ ]:
import numpy as np, pandas as pd, gc
import cv2, matplotlib.pyplot as plt
import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
print('RAPIDS',cuml.__version__)
print('TF',tf.__version__)

In [ ]:
# RESTRICT TENSORFLOW TO 1GB OF GPU RAM
# SO THAT WE HAVE 15GB RAM FOR RAPIDS
LIMIT = 1
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*LIMIT)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    #print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    print(e)
print('We will restrict TensorFlow to max %iGB GPU RAM'%LIMIT)
print('then RAPIDS can use %iGB GPU RAM'%(16-LIMIT))

# Load Train Data
First we load the train data and create a target column of ground truths to help us compute CV score. Note how the variable `COMPUTE_CV` will change to `False` when we **submit** this notebook but it is `True` now because you are reading a **commit** notebook.

In [ ]:
COMPUTE_CV = True

test = pd.read_csv('../input/shopee-product-matching/test.csv')
if len(test)>3: COMPUTE_CV = False
else: print('this submission notebook will compute CV score, but commit notebook will not')

In [ ]:
train = pd.read_csv('../input/shopee-product-matching/train.csv')
tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
train['target'] = train.label_group.map(tmp)
print('train shape is', train.shape )
train.head()

# Compute Baseline CV Score
A baseline is to predict all items with the same `image_phash` as being duplicate. Let's calcuate the CV score for this submission.

In [ ]:
tmp = train.groupby('image_phash').posting_id.agg('unique').to_dict()
train['oof'] = train.image_phash.map(tmp)

In [ ]:
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.target,row[col]) )
        return 2*n / (len(row.target)+len(row[col]))
    return f1score

In [ ]:
train['f1'] = train.apply(getMetric('oof'),axis=1)
print('CV score for baseline =',train.f1.mean())

# Compute RAPIDS Model CV and Infer Submission
We will now use image embeddings, text embeddings, and phash to create a better model with better CV. We will also infer submission csv.

Note how the variable `COMPUTE_CV` is only `True` when we **commit** this notebook. Right now you are reading a **commit** notebook, so we see test replaced with train and computed CV score. When we **submit** this notebook, the variable `COMPUTE_CV` will be `False` and the **submit** notebook will **not** compute CV. Instead it will load the real test dataset with 70,000 rows and find duplicates in the real test dataset.

In [ ]:
if COMPUTE_CV:
    test = pd.read_csv('../input/shopee-product-matching/train.csv')
    test_gf = cudf.DataFrame(test)
    print('Using train as test to compute CV (since commit notebook). Shape is', test_gf.shape )
else:
    test = pd.read_csv('../input/shopee-product-matching/test.csv')
    test_gf = cudf.read_csv('../input/shopee-product-matching/test.csv')
    print('Test shape is', test_gf.shape )
test_gf.head()

# Use Image Embeddings
To prevent memory errors, we will compute image embeddings in chunks. And we will find similar images with RAPIDS cuML KNN in chunks.

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, df, img_size=224, batch_size=8, path=''): 
        self.df = df
        self.img_size = img_size
        self.batch_size = batch_size
        self.path = path
        self.indexes = np.arange( len(self.df) )
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        ct = len(self.df) // self.batch_size
        ct += int(( (len(self.df)) % self.batch_size)!=0)
        return ct

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X = self.__data_generation(indexes)
        return X
            
    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' 
        X = np.zeros((len(indexes),self.img_size,self.img_size,3),dtype='float32')
        df = self.df.iloc[indexes]
        for i,(index,row) in enumerate(df.iterrows()):
            img = cv2.imread(self.path+row.image)
            X[i,] = cv2.resize(img,(self.img_size,self.img_size)) #/128.0 - 1.0
        return X

# resnet

In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import PIL
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras import applications, layers, losses, optimizers, Model
from tensorflow.keras.models import Sequential
import keras.backend as K
from sklearn.model_selection import train_test_split
from textwrap import wrap
import tensorflow as tf

224, 224, 3

In [ ]:
WGT = '../input/resnet50/resnet.h5'
IMAGENET='../input/tf-keras-resnet/resnet50_notop.h5'

In [ ]:
model_resnet = Sequential()
pretrained_model = applications.ResNet50(
    include_top=False,
    weights=IMAGENET,
    input_shape=(224, 224, 3),
    pooling='avg',
    classes=11014,
    classifier_activation='softmax',
)

In [ ]:
for layer in pretrained_model.layers:
    layer.trainable = False

model_resnet.add(pretrained_model)
# add batch normalization layer
model_resnet.add(BatchNormalization())
# flatten the output
model_resnet.add(Flatten())
# add dropout layer
model_resnet.add(Dropout(0.5))
# add linear layer for reduction
model_resnet.add(Dense(512, activation='linear'))
# add classification layer
model_resnet.add(Dense(11014, activation='softmax'))


In [ ]:
BASE = '../input/shopee-product-matching/test_images/'
if COMPUTE_CV: BASE = '../input/shopee-product-matching/train_images/'

#WGT = '../input/resnet50/resnet.h5'
#model = EfficientNetB0(weights=WGT,include_top=False, pooling='avg', input_shape=None)

embeds = []
CHUNK = 1024*4

print('Computing image embeddings...')
CTS = len(test)//CHUNK
if len(test)%CHUNK!=0: CTS += 1
for i,j in enumerate( range( CTS ) ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(test))
    print('chunk',a,'to',b)
    
    test_gen = DataGenerator(test.iloc[a:b], batch_size=32, path=BASE)
    image_embeddings = model_resnet.predict(test_gen,verbose=1,use_multiprocessing=True, workers=4)
    embeds.append(image_embeddings)

    #if i>=1: break
    
del model_resnet
_ = gc.collect()
image_embeddings = np.concatenate(embeds)
print('image embeddings shape',image_embeddings.shape)

In [ ]:
KNN = 50
if len(test)==3: KNN = 2
model_resnet = NearestNeighbors(n_neighbors=KNN)
model_resnet.fit(image_embeddings)

In [ ]:
preds_image = []
CHUNK = 1024*4

print('Finding similar images...')
CTS = len(image_embeddings)//CHUNK
if len(image_embeddings)%CHUNK!=0: CTS += 1
for j in range( CTS ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(image_embeddings))
    print('chunk',a,'to',b)
    distances, indices = model_resnet.kneighbors(image_embeddings[a:b,])
    
    for k in range(b-a):
        IDX = np.where(distances[k,]<6.0)[0]
        IDS = indices[k,IDX]
        o = test.iloc[IDS].posting_id.values
        preds_image.append(o)
        
del model_resnet, distances, indices, image_embeddings, embeds
_ = gc.collect()

In [ ]:
test['preds_image'] = preds_image
test.head()

# Use Text Embeddings
To prevent memory errors, we will find similar titles in chunks. To faciliate this, we will use cosine similarity between text embeddings instead of KNN.

In [ ]:
print('Computing text embeddings...')
model_resnet = TfidfVectorizer(stop_words='english', binary=True, max_features=25_000)
text_embeddings = model_resnet.fit_transform(test_gf.title).toarray()
print('text embeddings shape',text_embeddings.shape)

In [ ]:
preds_text = []
CHUNK = 1024*4

print('Finding similar titles...')
CTS = len(test)//CHUNK
if len(test)%CHUNK!=0: CTS += 1
for j in range( CTS ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(test))
    print('chunk',a,'to',b)
    
    # COSINE SIMILARITY DISTANCE
    cts = cupy.matmul( text_embeddings, text_embeddings[a:b].T).T
    
    for k in range(b-a):
        IDX = cupy.where(cts[k,]>0.7)[0]
        o = test.iloc[cupy.asnumpy(IDX)].posting_id.values
        preds_text.append(o)
        
del model_resnet, text_embeddings
_ = gc.collect()

In [ ]:
test['preds_text'] = preds_text
test.head()

# Use Phash Feature
We will predict all items with the same phash as duplicates

In [ ]:
tmp = test.groupby('image_phash').posting_id.agg('unique').to_dict()
test['preds_phash'] = test.image_phash.map(tmp)
test.head()

# Compute CV Score
This simple model scores a high CV of 0.700+!

CV

In [ ]:
test.head()

# image

In [ ]:
if COMPUTE_CV:
    tmp = test.groupby('label_group').posting_id.agg('unique').to_dict()
    test['target'] = test.label_group.map(tmp)
    
    test['oof'] = test['preds_image']
    test['f1'] = test.apply(getMetric('oof'),axis=1)
    print('CV Score =', test.f1.mean() )

# text

In [ ]:
if COMPUTE_CV:
    tmp = test.groupby('label_group').posting_id.agg('unique').to_dict()
    test['target'] = test.label_group.map(tmp)
    
    test['oof'] = test['preds_text']
    test['f1'] = test.apply(getMetric('oof'),axis=1)
    print('CV Score =', test.f1.mean() )

# phash

In [ ]:
if COMPUTE_CV:
    tmp = test.groupby('label_group').posting_id.agg('unique').to_dict()
    test['target'] = test.label_group.map(tmp)
    
    test['oof'] = test['preds_phash']
    test['f1'] = test.apply(getMetric('oof'),axis=1)
    print('CV Score =', test.f1.mean() )

# Combine

In [ ]:
def combine_for_sub(row):
    x = np.concatenate([row.preds_text,row.preds_image, row.preds_phash])
    return ' '.join( np.unique(x) )

def combine_for_cv(row):
    x = np.concatenate([row.preds_text,row.preds_image, row.preds_phash])
    return np.unique(x)

In [ ]:
if COMPUTE_CV:
    tmp = test.groupby('label_group').posting_id.agg('unique').to_dict()
    test['target'] = test.label_group.map(tmp)
    test['oof'] = test.apply(combine_for_cv,axis=1)
    test['f1'] = test.apply(getMetric('oof'),axis=1)
    print('CV Score =', test.f1.mean() )

test['matches'] = test.apply(combine_for_sub,axis=1)

# Write Submission CSV
In this notebook, the submission file below looks funny containing train information. But when we submit this notebook, the size of `test.csv` dataframe will be longer than 3 rows and the variable `COMPUTE_CV` will subsequently set to `False`. Then our submission notebook will compute the correct matches using the real test dataset and our submission csv for LB will be ok.

In [ ]:
test[['posting_id','preds_text']].to_csv('submission.csv',index=False)
sub = pd.read_csv('submission.csv')
sub.head()